In [2]:
import numpy as np
import pandas as pd
import torch
import pickle
import matplotlib.pyplot as plt
from lpne.models import DcsfaNmf
import os, sys

MODEL_FILE = "/hpc/home/mk423/Anxiety/FullDataWork/Models/Final_mt_Model_500_epochs.pt"
DATA_PATH = "/work/mk423/FoberPho_EPM_Anxiety/"
PROJECT_PATH = "/work/mk423/FoberPho_EPM_Anxiety/"
LPNE_PATH = "/work/mk423/lpne-data-analysis/"

model = torch.load(MODEL_FILE,map_location="cpu")
model.device="cpu"

sys.path.append(LPNE_PATH)

import data_tools

In [15]:
X_psd, X_coh, X_gc, labels = data_tools.load_data(DATA_PATH + "FiberPho_EPM_from_Hunter.mat",
                                                  feature_list = ["power","coherence","granger"])

version saveFeatures_1.2 used to calcuate power features
version saveFeatures_1.2 used to calcuate coherence features
version saveFeatures_1.2 used to calcuate granger features
Version preprocessData_1.0 of preprocessing used
Make sure feature versions listed above match those used for any other dataset in the same project


In [20]:
s = model.project(np.hstack([X_psd*10,X_coh,X_gc]))
coeffs = np.abs(model.classifier[0].weight[0].detach().cpu().numpy())
agg_score = s[:,:3] @ coeffs

In [22]:
saveDict = {
    "net 1":s[:,0],
    "net 2":s[:,1],
    "net 3":s[:,2],
    "agg score":agg_score,
    "time":labels['windows']['time'],
    "mouse":labels['windows']['mouse'],
    "expDate":labels['windows']['expDate'],
}

df = pd.DataFrame.from_dict(saveDict)
df.to_csv(PROJECT_PATH + "FoberPho_EPM_Anxiety_Projection.csv")

In [21]:
labels['windows'].keys()

dict_keys(['mouse', 'expDate', 'time'])

In [23]:
df

,net 1,net 2,net 3,agg score,time,mouse,expDate
0,0.075902,0.085406,0.044612,2.199738,1,Mouse57461,110823
1,0.126518,0.220316,0.010726,4.920426,2,Mouse57461,110823
2,0.078984,0.090500,0.003551,2.285154,3,Mouse57461,110823
3,0.084782,0.114406,0.029680,2.759445,4,Mouse57461,110823
4,0.100374,0.083204,0.002404,2.372173,5,Mouse57461,110823
...,...,...,...,...,...,...,...
2631,0.094966,0.221003,0.046275,4.648554,408,Mouse57475,110823
2632,0.104525,0.148928,0.028398,3.527501,409,Mouse57475,110823
2633,0.053269,0.106117,0.007840,2.296788,410,Mouse57475,110823
2634,0.053934,0.078431,0.051762,1.873453,411,Mouse57475,110823
